In [1]:
import pandas as pd
import numpy as np

In [2]:
dfp = pd.read_json('reddit_pat.json', lines=True)
dfk = pd.read_json('reddit_kv.json', lines=True)

In [3]:
print(dfp.shape)
display(dfp.head())
print(dfk.shape)
display(dfk.head())

(2450, 5)


,age,num_comments,score,text,timestamp
0,6 hours ago,1067,16886,Pakistan will release Indian Pilot Abhinandan ...,2019-02-28 11:16:12
1,10 hours ago,5338,43583,Trump-Kim talks end 'without agreement',2019-02-28 07:01:12
2,7 hours ago,1099,4036,UN Council: Israel Intentionally Shot Children...,2019-02-28 10:47:58
3,1 hour ago,89,592,Israel's Attorney General Avichai Mendelblit a...,2019-02-28 16:05:00
4,2 hours ago,175,663,Cohen connects the dots on what Trump knew abo...,2019-02-28 15:08:29


(2650, 5)


,age,num_comments,score,text,timestamp
0,5 hours ago,1588,21720,China Killing Prisoners To Harvest Organs For ...,2019-06-19 11:49:08
1,4 hours ago,402,2661,Muslim family dragged out of Belgian embassy i...,2019-06-19 12:05:31
2,11 hours ago,3320,46977,Women outperform men after Japan medical schoo...,2019-06-19 05:51:44
3,6 hours ago,202,1474,MH17 crash: Investigators 'to charge four with...,2019-06-19 10:50:51
4,11 hours ago,1336,2665,Iranian official calls on world to unite again...,2019-06-19 05:09:15


In [4]:
reddit_combined = pd.concat([dfk,dfp], ignore_index=True)
reddit_combined.shape

(5100, 5)

In [5]:
reddit_combined.tail()

,age,num_comments,score,text,timestamp
5095,1 hour ago,20,117,"‘Kremlin Used’ NRA to Help Trump in 2016, Says...",2018-05-16 16:55:22
5096,23 hours ago,4356,31969,N. Korea cancels Wednesday's high-level talks ...,2018-05-15 18:31:54
5097,9 hours ago,62,464,Erdoğan ends UK state visit by calling jailed ...,2018-05-16 09:13:12
5098,16 hours ago,126,2007,Oil companies have played a major role in clim...,2018-05-16 01:17:21
5099,13 hours ago,492,1060,Nikki Haley denies Gaza violence is related to...,2018-05-16 04:31:37


Indexes are properly assigned

In [6]:
reddit_combined.drop_duplicates(subset='text', keep='first').shape     # Keeps the first(lowest index)
                                                                    # Which is the later(time) archive

(4843, 5)

In [7]:
reddit_trimmed=reddit_combined.drop_duplicates(subset='text',keep='first')

In [8]:
reddit_trimmed.to_json('reddit_combined.json')

In [91]:
df = pd.read_json('reddit_combined.json').sort_index()
df.head()

,age,num_comments,score,text,timestamp
0,5 hours ago,1588,21720,China Killing Prisoners To Harvest Organs For ...,2019-06-19 11:49:08
1,4 hours ago,402,2661,Muslim family dragged out of Belgian embassy i...,2019-06-19 12:05:31
2,11 hours ago,3320,46977,Women outperform men after Japan medical schoo...,2019-06-19 05:51:44
3,6 hours ago,202,1474,MH17 crash: Investigators 'to charge four with...,2019-06-19 10:50:51
4,11 hours ago,1336,2665,Iranian official calls on world to unite again...,2019-06-19 05:09:15


In [92]:
df.describe()

,num_comments,score
count,4843.000000,4843.000000
mean,694.733636,8496.281644
std,1409.812347,16760.184113
min,0.000000,17.000000
25%,44.000000,241.000000
50%,165.000000,1098.000000
75%,667.500000,5786.000000
max,27532.000000,142436.000000


In [93]:
df.isna().sum()

age             0
num_comments    0
score           0
text            0
timestamp       0
dtype: int64

In [94]:
df.age.unique()

array(['5 hours ago', '4 hours ago', '11 hours ago', '6 hours ago',
       '2 hours ago', '21 hours ago', '3 hours ago', '16 hours ago',
       '1 day ago', '10 hours ago', '23 hours ago', '13 hours ago',
       '1 hour ago', '14 hours ago', '18 hours ago', '15 hours ago',
       '9 hours ago', '20 hours ago', '17 hours ago', '19 hours ago',
       '12 hours ago', '7 hours ago', '22 hours ago', '8 hours ago',
       '50 minutes ago', '41 minutes ago', '43 minutes ago',
       '45 minutes ago', '47 minutes ago', '22 minutes ago',
       '56 minutes ago', '59 minutes ago', '18 minutes ago',
       '35 minutes ago', '34 minutes ago', '55 minutes ago',
       '29 minutes ago', '33 minutes ago', '54 minutes ago',
       '39 minutes ago', '28 minutes ago', '40 minutes ago',
       '31 minutes ago', '46 minutes ago'], dtype=object)

In [95]:
def clean_age(text):
    """Compute age in minutes"""
    if 'day' in text:
        return 1440
    if 'hour' in text:
        return int(text[:2]) * 60
    else:
        return int(text[:2])

In [96]:
df.loc[:,'weekday_posted'] = df['timestamp'].map(lambda x: x.weekday())
df.loc[:,'hour_posted'] = df['timestamp'].map(lambda x: x.hour)
df['age'] = df['age'].map(clean_age)

In [97]:
df.loc[:,'log_score'] = np.log(df['score'])
df.loc[:,'log_comments'] = np.log(df['num_comments'] + 0.01)

In [98]:
df.head()

,age,num_comments,score,text,timestamp,weekday_posted,hour_posted,log_score,log_comments
0,300,1588,21720,China Killing Prisoners To Harvest Organs For ...,2019-06-19 11:49:08,2,11,9.985989,7.370237
1,240,402,2661,Muslim family dragged out of Belgian embassy i...,2019-06-19 12:05:31,2,12,7.886457,5.996477
2,660,3320,46977,Women outperform men after Japan medical schoo...,2019-06-19 05:51:44,2,5,10.757413,8.107723
3,360,202,1474,MH17 crash: Investigators 'to charge four with...,2019-06-19 10:50:51,2,10,7.295735,5.308317
4,660,1336,2665,Iranian official calls on world to unite again...,2019-06-19 05:09:15,2,5,7.887959,7.197443


In [15]:
from sklearn.model_selection import train_test_split

In [13]:
text_body = df.text

In [10]:
from sklearn.feature_extraction.text import  TfidfVectorizer

In [11]:
tfidf = TfidfVectorizer()

In [14]:
tfidf.fit(text_body)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)